In [1]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

In [2]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [3]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [4]:
# Send a Copy of the Model to Alice and Bob
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [5]:
# Train Bob's and Alice's Models (in parallel)
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.6489) Alice:tensor(9.0547)
Bob:tensor(0.1476) Alice:tensor(0.0910)
Bob:tensor(0.0338) Alice:tensor(0.0185)
Bob:tensor(0.0079) Alice:tensor(0.0150)
Bob:tensor(0.0020) Alice:tensor(0.0124)
Bob:tensor(0.0006) Alice:tensor(0.0104)
Bob:tensor(0.0003) Alice:tensor(0.0086)
Bob:tensor(0.0002) Alice:tensor(0.0072)
Bob:tensor(0.0002) Alice:tensor(0.0060)
Bob:tensor(0.0001) Alice:tensor(0.0050)


In [6]:
# Send Both Updated Models to a Secure Worker
alices_model.move(secure_worker)

In [7]:
bobs_model.move(secure_worker)

In [8]:
# Average the Models
with torch.no_grad():   # 将所有计算得出的tensor的requires_grad都自动设置为False
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [9]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0032) Alice:tensor(0.0127)
Bob:tensor(0.0051) Alice:tensor(0.0072)
Bob:tensor(0.0058) Alice:tensor(0.0037)
Bob:tensor(0.0057) Alice:tensor(0.0019)
Bob:tensor(0.0052) Alice:tensor(0.0010)
Bob:tensor(0.0045) Alice:tensor(0.0006)
Bob:tensor(0.0038) Alice:tensor(0.0003)
Bob:tensor(0.0031) Alice:tensor(0.0002)
Bob:tensor(0.0025) Alice:tensor(0.0001)
Bob:tensor(0.0020) Alice:tensor(7.9550e-05)


In [10]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [11]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1291],
        [0.1079],
        [0.8634],
        [0.8422]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0719)
